In [2]:
import os
# import time
# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
os.chdir(r'C:\Users\Dragon\Desktop')

In [4]:
# start = time.time()

# - 导入待派券名单
with pd.ExcelFile(r'C:\Users\Dragon\Desktop\工作簿8.xlsx') as xlsx:
    df_1 = pd.read_excel(xlsx,'1')
    df_2 = pd.read_excel(xlsx,'2')
#     df_cg = pd.read_excel(xlsx,'存管回款')
#     df_quan = pd.read_excel(xlsx,'券')

In [5]:
df = pd.merge(df_1,df_2,
             on = '手机号',
             how = 'left')

In [6]:
df.shape

(267972, 8)

In [7]:
df_1.shape

(267972, 7)

In [8]:
df.to_excel('1111.xlsx')

In [ ]:
# - 导入已派券名单
df_rcd = pd.read_excel(r'D:\小鸡理财\百度云同步盘\小鸡理财\每日数据\派券\已派券记录.xlsx')

print('派券名单导入完成!\n')

In [ ]:
# - 合并回款表
df_hk = df_jd_bj.append([df_jd_lx,df_cg], ignore_index=True,sort=True)

# - 整理回款表
df_hk['发放时间'] = pd.to_datetime(df_hk['预计本次发放时间'])
del df_hk['预计本次发放时间']

In [ ]:
dtnow = pd.to_datetime('today')
day_of_week = dtnow.day_name()
hour_of_day = int(dtnow.hour)
print('Today is ',day_of_week)

if day_of_week == 'Friday':
	print('---happy 周末 !---\n')

	timediff = pd.Timedelta(1,unit='d')
	dt_ff_1 = dtnow + timediff
	dt_ff_1 = dt_ff_1.strftime('%Y.%m.%d')

	timediff = pd.Timedelta(2,unit='d')
	dt_ff_2 = dtnow + timediff
	dt_ff_2 = dt_ff_2.strftime('%Y.%m.%d')

	timediff = pd.Timedelta(3,unit='d')
	dt_ff_3 = dtnow + timediff
	dt_ff_3 = dt_ff_3.strftime('%Y.%m.%d')

	df_hk_today = df_hk[(df_hk['发放时间'] == dt_ff_1) | (df_hk['发放时间'] == dt_ff_2) | (df_hk['发放时间'] == dt_ff_3)]

	dt_ff = str(dt_ff_1) + '-' + str(dt_ff_3)

else:
	print('---平时 is choosed!---\n')
	timediff = pd.Timedelta(1,unit='d')
	dt_ff = dtnow + timediff
	dt_ff = dt_ff.strftime('%Y.%m.%d')
	df_hk_today = df_hk[(df_hk['发放时间'] == dt_ff)]

In [ ]:
#归类
class_pai = dict({'6月标':'派券2',
                  '12月标':'派券2',
                  360:'派券2',
                  180:'派券2',
                  90:'派券1',
                  30:'派券1'})

df_hk_today = df_hk_today.copy()
df_hk_today['分类'] = df_hk_today.loc[:,'投资期限'].map(class_pai)

In [ ]:
df_hk_today.head(10)

In [ ]:
hk_pp = df_hk_today['会员名'].nunique()

In [ ]:
hk_money = round(df_hk_today['本次发放金额'].sum() / 10000)

In [ ]:
hk_money

In [ ]:
print('{} 回款信息:共回款 {:.0f} 人，合计回款金额 {:.0f} 万元'.format(dt_ff,hk_pp,hk_money))

In [ ]:
# - 合并计算，排除当日回款金额小于500元
#排除总金额<500
gp_hk = df_hk_today.groupby('会员名',as_index=False)['本次发放金额'].sum()
l500 = gp_hk[gp_hk['本次发放金额'] >= 500]['会员名']
# l500
df_hk_today_res_1 = df_hk_today[df_hk_today['会员名'].isin(l500)]

In [ ]:
pp_500_num = len(gp_hk[gp_hk['本次发放金额'] < 500]['会员名'])

In [ ]:
pp_500

In [ ]:
# - 排除有3张券以上的人
df_quan['mark'] = 1
gp_quan = df_quan.groupby('会员名',as_index=False)['mark'].sum()
q3 = gp_quan[gp_quan['mark'] >= 3]['会员名']
df_hk_today_res_2 = df_hk_today_res_1[~df_hk_today_res_1['会员名'].isin(q3)]

In [ ]:
df_q3 = df_hk_today[df_hk_today['会员名'].isin(q3)]

In [ ]:
df_q3.head()

In [ ]:
df_q3_num = df_q3['会员名'].nunique()

In [ ]:
# - 排除已经派了4次的人
df_rcd['mark'] = 1
gp_rcd = df_rcd.groupby('会员名',as_index=False)['mark'].sum()
p4 = gp_rcd[gp_rcd['mark'] >= 4]
df_hk_today_res_3 = df_hk_today_res_2[~df_hk_today_res_2['会员名'].isin(p4['会员名'])]

In [ ]:
df_p4 = df_hk_today[df_hk_today['会员名'].isin(p4['会员名'])]

In [ ]:
df_p4_num = df_p4['会员名'].nunique()

In [ ]:
# - 确定要派哪类券
gp_class = df_hk_today_res_3.groupby(['会员名','真实姓名','分类'])['本次发放金额'].sum().unstack()
gp_class.fillna(0,inplace=True)

if len(gp_class.columns) == 1:
	gp_class['派券分类'] = gp_class.columns[0]
else:
	gp_class['派券分类'] = np.where(gp_class['派券2'] == 0,'派券1','派券2')

print('class_ok!\n')

# class_time = time.time()
# print('class_time:{} s\n'.format(class_time-filter_time))

# - 查找用户名等信息
res_temp = gp_class.copy()
res_temp = res_temp.reset_index()
res = res_temp.loc[:,['会员名','真实姓名','派券分类']]


In [ ]:
p_num = res['会员名'].nunique()

In [ ]:
print('其中回款金额小于500元 {} 人，拥有3张券以上的 {} 人，已经发放4次券包的 {} 人。\n{} 派券 {} 人！'.format(
        pp_500,df_q3_num,df_p4_num,dt_ff,p_num))

In [ ]:
print('''
TO 财务部：

附件是 {} 新规则派券名单
	'''.format(dt_ff))
print('{} 回款信息:共回款 {:.0f} 人，合计回款金额 {:.0f} 万元。'.format(dt_ff,hk_pp,hk_money))
print('其中回款金额小于500元 {} 人，拥有3张券以上的 {} 人，已经发放4次券包的 {} 人。\n\n{} 派券 {} 人！'.format(
        pp_500_num,df_q3_num,df_p4_num,dt_ff,p_num))

In [ ]:
%timeit df = pd.read_csv(r"D:\VDownload\NYC Yellow Taxi data\fhv_tripdata_2018-03.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
with pd.ExcelFile(r'C:\Users\Xiaoji\Desktop\渠道表.xlsx') as xlsx:
    df_tz = pd.read_excel(xlsx,'投资')
    df_zc = pd.read_excel(xlsx,'注册')
#     df_yj = pd.read_excel(xlsx,'佣金')
#     df_vip = pd.read_excel(xlsx,'VIP变动')
#     df_quan = pd.read_excel(xlsx,'券')
#     df_zht = pd.read_excel(xlsx,'智慧投')

In [ ]:
df = pd.merge(df_tz,df_zc,
             on='手机号',
             how='left')

In [ ]:
df.to_excel('1030.xlsx')

In [ ]:
df = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\green_tripdata_2013-08_2.csv',
                     sep=r'(?:,|\t|\r|\f|\v)',
                    engine='python')

In [ ]:
df.head()

In [ ]:
reader = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\green_tripdata_2013-08_2.csv',
                     sep=',',nrows=0)
#                      chunksize=10)

In [ ]:
reader.info()

In [ ]:
reader = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\yellow_tripdata_2012-03.csv',
                     sep=',',
                     nrows=0)

In [ ]:
reader.info()

In [ ]:
reader.info()

In [ ]:
reader

In [ ]:
for chunk in reader:
    print(chunk.shape[0])
#     print(chunk.head())
    print('='*50)

In [ ]:
n = 1

In [ ]:
while n:
    

In [ ]:
l = [1,2,3]

In [ ]:
print(reader.next())

In [ ]:
import re

In [ ]:
m = re.match(r'a([bcd]*)e','abce')

In [ ]:
m.string

In [ ]:
sep=r'(?:,|\s+)'

In [ ]:
l = [1,5,12,2]
s = ''

for i in l:
    s = s + str(i)
print(s)

In [ ]:
color_list_1 = set(["White", "Black", "Red"])
color_list_2 = set(["Red", "Green"])

print(color_list_1.difference(color_list_2))

In [ ]:
def gcd(a, b):
    return a if b == 0 else gcd(b, a%b)

gcd(336,360)

In [ ]:
def lcm(a, b):
    return int(a * b / gcd(a, b))

print(lcm(15, 17))

In [ ]:
char = '@'

l = [2,3,6,5]

for i in l:
    print(char * i)

In [2]:
s = pd.Series([1,2,3,4,5])
s

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [3]:
l = list(s)
l

[1, 2, 3, 4, 5]

In [4]:
s.tolist()

[1, 2, 3, 4, 5]

In [5]:
s1 = pd.Series([2, 4, 6, 8, 10])
s2 = pd.Series([1, 3, 5, 7, 9])

In [6]:
s1+s2

0     3
1     7
2    11
3    15
4    19
dtype: int64

In [7]:
s1-s2

0    1
1    1
2    1
3    1
4    1
dtype: int64

In [8]:
s1 * s2

0     2
1    12
2    30
3    56
4    90
dtype: int64

In [9]:
s1 / s2

0    2.000000
1    1.333333
2    1.200000
3    1.142857
4    1.111111
dtype: float64

In [10]:
s = pd.Series({'a': 100, 'b': 200, 'c': 300, 'd': 400, 'e': 800})

In [11]:
s

a    100
b    200
c    300
d    400
e    800
dtype: int64

In [13]:
n = np.array([10, 20, 30, 40, 50])

In [14]:
n

array([10, 20, 30, 40, 50])

In [15]:
s = pd.Series(n)

In [16]:
s

0    10
1    20
2    30
3    40
4    50
dtype: int32

In [17]:
s1 = pd.Series(['100', '200', 'python', '300.12', '400'])

In [19]:
s2 = pd.to_numeric(s1, errors='coerce')

In [20]:
s2

0    100.00
1    200.00
2       NaN
3    300.12
4    400.00
dtype: float64

In [21]:
d = {'col1': [1, 2, 3, 4, 7, 11], 'col2': [4, 5, 6, 9, 5, 0], 'col3': [7, 5, 8, 12, 1,11]}
df = pd.DataFrame(data=d)

In [22]:
s = df.col1

In [23]:
s

0     1
1     2
2     3
3     4
4     7
5    11
Name: col1, dtype: int64

In [24]:
s = pd.Series([
    ['Red', 'Green', 'White'],
    ['Red', 'Black'],
    ['Yellow']])

In [25]:
s

0    [Red, Green, White]
1           [Red, Black]
2               [Yellow]
dtype: object

In [28]:
s.apply(pd.Series).stack().reset_index(drop=True)

0       Red
1     Green
2     White
3       Red
4     Black
5    Yellow
dtype: object

In [29]:
s = pd.Series(['100', '200', 'python', '300.12', '400'])
s.sort_values()

0       100
1       200
3    300.12
4       400
2    python
dtype: object

In [31]:
s = pd.Series([0, 1,2,3,4,5,6,7,8,9,10])
print("Original Data Series:")
print(s)
print("\nSubset of the above Data Series:")
n = 3
new_s = s[s < n]
print(new_s)

Original Data Series:
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
dtype: int64

Subset of the above Data Series:
0    0
1    1
2    2
dtype: int64


In [32]:
s = pd.Series(data = [1,2,3,4,5,6,7,8,9,5,3])

In [35]:
s.std()

2.522624895547565

In [39]:
df = pd.DataFrame( {'X':[78,85,96,80,86], 'Y':[84,94,89,83,86],'Z':[86,97,96,72,83]})

In [40]:
df

,X,Y,Z
0,78,84,86
1,85,94,97
2,96,89,96
3,80,83,72
4,86,86,83


In [41]:
exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'],
'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

df = pd.DataFrame(exam_data,index=labels)
df

,name,score,attempts,qualify
a,Anastasia,12.5,1,yes
b,Dima,9.0,3,no
c,Katherine,16.5,2,yes
d,James,NaN,3,no
e,Emily,9.0,2,no
f,Michael,20.0,3,yes
g,Matthew,14.5,1,yes
h,Laura,NaN,1,no
i,Kevin,8.0,2,no
j,Jonas,19.0,1,yes


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, a to j
Data columns (total 4 columns):
name        10 non-null object
score       8 non-null float64
attempts    10 non-null int64
qualify     10 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 400.0+ bytes


In [43]:
df.iloc[:3,:]

,name,score,attempts,qualify
a,Anastasia,12.5,1,yes
b,Dima,9.0,3,no
c,Katherine,16.5,2,yes


In [45]:
df.loc[df.attempts > 2]

,name,score,attempts,qualify
b,Dima,9.0,3,no
d,James,NaN,3,no
f,Michael,20.0,3,yes


In [46]:
df.shape

(10, 4)

In [47]:
df.isnull()

,name,score,attempts,qualify
a,False,False,False,False
b,False,False,False,False
c,False,False,False,False
d,False,True,False,False
e,False,False,False,False
f,False,False,False,False
g,False,False,False,False
h,False,True,False,False
i,False,False,False,False
j,False,False,False,False
